In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Trends with the GOZCARDS Dataset

Here we calculate trends using the GOZCARDS dataset by regressing to the VMR monthly zonal means using seasonal terms in our predictors.

In [ ]:
import xarray as xr
import numpy as np
from LOTUS_regression.regression import regress_all_bins
from LOTUS_regression.predictors.seasonal import add_seasonal_components
from LOTUS_regression.predictors import load_data

The GOZCARDS data is in multiple NetCDF4 files.  Load them all in and concatenate on the time dimension.  Also only take values in the latitude range -60 to 60.

In [ ]:
GOZCARDS_FILES = r'/home/runner/work/lotus-regression/lotus-regression/test_data//GOZCARDS/*.nc4'
data = xr.decode_cf(xr.open_mfdataset(GOZCARDS_FILES, combine='nested', concat_dim='time', group='Merged', engine='netcdf4'))

data = data.loc[dict(lat=slice(-60, 60))]

print(data)

Load some standard predictors and add a constant

In [ ]:
predictors = load_data('pred_baseline_pwlt.csv')

print(predictors.columns)

Currently, our predictors have no seasonal dependence.  Add in some seasonal terms with different numbers of Fourier components.

In [ ]:
predictors = add_seasonal_components(predictors, {'constant': 4, 'linear_pre': 2, 'linear_post': 2, 'qboA': 2, 'qboB': 2})

print(predictors[:5])

Perform the regression and convert the coefficients to percent anomaly.

In [ ]:
data = data.sel(lev=slice(100, 0.2)).sel(lat=slice(-65, 65))   # remove bins without data
results = regress_all_bins(predictors, data['average'], tolerance=0.1)

# Convert to ~ percent
results /= data['average'].mean(dim='time')
results *= 100

print(results)

In [ ]:
import LOTUS_regression.plotting.trends as trends
trends.pre_post_with_confidence(results, x='lat', y='lev', ylim=(100, 0.5), log_y=True, figsize=(16, 6),
                                x_label='Latitude [$^\circ$]', y_label='Pressure [hPa]', pre_title='Pre 1997',
                                post_title='Post 1997')